In [1]:
import pandas as pd
import plotly.graph_objects as go
import streamlit as st
import plotly.express as px
from mainnet_launch.constants import (
    AutopoolConstants,
    STATS_CALCULATOR_REGISTRY,
    ALL_AUTOPOOLS,
    ETH_CHAIN,
    ROOT_PRICE_ORACLE,
    ChainData,
    WETH,
)

from mainnet_launch.database.schema.full import (
    AutopoolDestinationStates,
    Destinations,
    DestinationStates,
    Blocks,
    DestinationTokenValues,
    Tokens,
    TokenValues,
)
from mainnet_launch.database.schema.postgres_operations import (
    merge_tables_as_df,
    TableSelector,
    get_full_table_as_df,
    natural_left_right_using_where,
)


df = natural_left_right_using_where(
    DestinationStates,
    Destinations,
    using=[Destinations.destination_vault_address],
    where_clause=Destinations.chain_id == 1,
)

2025-05-02 17:35:55,417 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-05-02 17:35:55,419 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 17:35:55,604 INFO sqlalchemy.engine.Engine select current_schema()
2025-05-02 17:35:55,605 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 17:35:55,796 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-05-02 17:35:55,798 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 17:35:55,989 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 17:35:55,990 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [2]:
px.line(df.pivot(index="block", columns="destination_vault_address", values="underlying_token_total_supply"))
# underlying token_total_supply looks right

In [3]:
import pandas as pd
import plotly.graph_objects as go
import streamlit as st
import plotly.express as px
from mainnet_launch.constants import (
    AutopoolConstants,
    STATS_CALCULATOR_REGISTRY,
    ALL_AUTOPOOLS,
    ETH_CHAIN,
    ROOT_PRICE_ORACLE,
    ChainData,
    WETH,
)

from mainnet_launch.database.schema.full import (
    AutopoolDestinationStates,
    Destinations,
    DestinationStates,
    Blocks,
    DestinationTokenValues,
    Tokens,
    TokenValues,
)
from mainnet_launch.database.schema.postgres_operations import (
    merge_tables_as_df,
    TableSelector,
)


def _fetch_tvl_by_destination(autopool: AutopoolConstants) -> pd.DataFrame:
    autopool_value_over_time_by_df = merge_tables_as_df(
        [
            TableSelector(
                table=AutopoolDestinationStates,
                select_fields=[AutopoolDestinationStates.owned_shares],
                join_on=None,
                row_filter=(AutopoolDestinationStates.autopool_vault_address == autopool.autopool_eth_addr),
            ),
            TableSelector(
                table=Destinations,
                select_fields=Destinations.underlying_symbol,
                join_on=(AutopoolDestinationStates.chain_id == Destinations.chain_id)
                & (AutopoolDestinationStates.destination_vault_address == Destinations.destination_vault_address),
            ),
            TableSelector(
                table=DestinationStates,
                select_fields=DestinationStates.price_per_share,
                join_on=(DestinationStates.chain_id == Destinations.chain_id)
                & (DestinationStates.destination_vault_address == Destinations.destination_vault_address),
            ),
            TableSelector(
                table=Blocks,
                select_fields=Blocks.datetime,
                join_on=(AutopoolDestinationStates.chain_id == Blocks.chain_id)
                & (AutopoolDestinationStates.block == Blocks.block),
            ),
        ],
        order_by=Blocks.datetime,
    )

    autopool_value_over_time_by_df["safe_tvl"] = (
        autopool_value_over_time_by_df["owned_shares"] * autopool_value_over_time_by_df["price_per_share"]
    )

    # group together destinations by the underlying token symbols
    # becuase more than one destiantion can be in the same vault
    autopool_value_over_time_by_df = (
        autopool_value_over_time_by_df.groupby(["datetime", "underlying_symbol"])["safe_tvl"].sum().reset_index()
    )
    return autopool_value_over_time_by_df.pivot(
        index="datetime", values="safe_tvl", columns="underlying_symbol"
    ).fillna(0)


def _fetch_tvl_by_asset(autopool: AutopoolConstants) -> pd.DataFrame:

    token_value_df = merge_tables_as_df(
        [
            TableSelector(
                table=Tokens,
                select_fields=[Tokens.symbol, Tokens.token_address],
                join_on=None,
            ),
            TableSelector(
                table=TokenValues,
                select_fields=[TokenValues.safe_price, TokenValues.block],
                join_on=(TokenValues.chain_id == Tokens.chain_id) & (TokenValues.token_address == Tokens.token_address),
            ),
            TableSelector(
                table=Blocks,
                select_fields=[
                    Blocks.datetime,
                ],
                join_on=(Blocks.chain_id == Tokens.chain_id) & (Blocks.block == TokenValues.block),
            ),
        ]
    )

    # this is too slow, it should be a view, add later, (prob a materialized view)
    # 20 seconds
    destinations_df = merge_tables_as_df(
        [
            TableSelector(
                table=AutopoolDestinationStates,
                select_fields=[
                    AutopoolDestinationStates.owned_shares,
                    AutopoolDestinationStates.destination_vault_address,
                    AutopoolDestinationStates.block,
                ],
                join_on=None,
                row_filter=((AutopoolDestinationStates.autopool_vault_address == autopool.autopool_eth_addr)),
            ),
            TableSelector(
                table=DestinationTokenValues,
                select_fields=[
                    DestinationTokenValues.token_address,
                    DestinationTokenValues.quantity,
                ],  # total amount of the token in the whole poool
                join_on=(DestinationTokenValues.chain_id == AutopoolDestinationStates.chain_id)
                & (
                    DestinationTokenValues.destination_vault_address
                    == AutopoolDestinationStates.destination_vault_address
                )
                & (DestinationTokenValues.block == AutopoolDestinationStates.block),
            ),
            TableSelector(
                table=DestinationStates,
                select_fields=[
                    DestinationStates.underlying_token_total_supply,
                    DestinationStates.price_per_share,
                    DestinationStates.safe_total_supply,
                ],  # lp token total supply
                join_on=(DestinationStates.chain_id == DestinationTokenValues.chain_id)
                & (DestinationStates.destination_vault_address == DestinationTokenValues.destination_vault_address)
                & (DestinationStates.block == DestinationTokenValues.block),
            ),
        ]
    )
    # 3m rows, that does not make any sense, is way to much
    return destinations_df, token_value_df


destinations_df, token_value_df = _fetch_tvl_by_asset(ALL_AUTOPOOLS[0])

2025-05-02 17:35:57,208 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 17:35:57,208 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-05-02 17:35:57,209 INFO sqlalchemy.engine.Engine [cached since 1.22s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x15fdecd90>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-05-02 17:35:57,209 INFO sqlalchemy.engine.Engine SELECT
    tokens.symbol,
    tokens.token_address,
    token_values.safe_price,
    token_values.block,
    blocks.datetime
FRO

In [4]:
destinations_df.block.value_counts()

block
20759464    59
21790905    59
21805232    59
21812397    59
21819559    59
            ..
21332578    59
21339732    59
21346894    59
21354047    59
22392426    59
Name: count, Length: 229, dtype: int64

In [5]:
dest_df = pd.merge(destinations_df, token_value_df, on=["block", "token_address"])
dest_df

,owned_shares,destination_vault_address,block,token_address,quantity,underlying_token_total_supply,price_per_share,safe_total_supply,symbol,safe_price,datetime
0,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20759464,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7077.280862,13157.451200,1.036523,12302.611124,WETH,1.0,2024-09-15 23:59:59+00:00
1,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20766617,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7091.767922,13157.496413,1.036163,12383.288934,WETH,1.0,2024-09-16 23:59:59+00:00
2,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20773761,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7042.071304,13157.496413,1.036462,12383.288934,WETH,1.0,2024-09-17 23:59:59+00:00
3,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20780916,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7145.366574,13268.458946,1.036491,12479.834854,WETH,1.0,2024-09-18 23:59:59+00:00
4,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20788084,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7069.241070,13268.526874,1.036501,12494.174709,WETH,1.0,2024-09-19 23:59:59+00:00
...,...,...,...,...,...,...,...,...,...,...,...
13506,110.472695,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22363838,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,110.472695,NaN,1.000000,NaN,WETH,1.0,2025-04-27 23:59:59+00:00
13507,273.545207,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22370998,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,273.545207,NaN,1.000000,NaN,WETH,1.0,2025-04-28 23:59:59+00:00
13508,179.975418,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22378150,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,179.975418,NaN,1.000000,NaN,WETH,1.0,2025-04-29 23:59:59+00:00
13509,188.715846,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22385293,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,188.715846,NaN,1.000000,NaN,WETH,1.0,2025-04-30 23:59:59+00:00


In [6]:
px.scatter(dest_df, x="safe_total_supply", y="underlying_token_total_supply", color="token_address")

In [7]:
dest_df["portion_ownership"] = dest_df["owned_shares"] / dest_df["underlying_token_total_supply"]

dest_df["proportional_value"] = dest_df["portion_ownership"] * dest_df["quantity"] * dest_df["safe_price"]

In [8]:
dest_df

,owned_shares,destination_vault_address,block,token_address,quantity,underlying_token_total_supply,price_per_share,safe_total_supply,symbol,safe_price,datetime,portion_ownership,proportional_value
0,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20759464,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7077.280862,13157.451200,1.036523,12302.611124,WETH,1.0,2024-09-15 23:59:59+00:00,0.0,0.0
1,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20766617,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7091.767922,13157.496413,1.036163,12383.288934,WETH,1.0,2024-09-16 23:59:59+00:00,0.0,0.0
2,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20773761,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7042.071304,13157.496413,1.036462,12383.288934,WETH,1.0,2024-09-17 23:59:59+00:00,0.0,0.0
3,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20780916,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7145.366574,13268.458946,1.036491,12479.834854,WETH,1.0,2024-09-18 23:59:59+00:00,0.0,0.0
4,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20788084,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7069.241070,13268.526874,1.036501,12494.174709,WETH,1.0,2024-09-19 23:59:59+00:00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13506,110.472695,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22363838,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,110.472695,NaN,1.000000,NaN,WETH,1.0,2025-04-27 23:59:59+00:00,NaN,NaN
13507,273.545207,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22370998,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,273.545207,NaN,1.000000,NaN,WETH,1.0,2025-04-28 23:59:59+00:00,NaN,NaN
13508,179.975418,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22378150,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,179.975418,NaN,1.000000,NaN,WETH,1.0,2025-04-29 23:59:59+00:00,NaN,NaN
13509,188.715846,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22385293,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,188.715846,NaN,1.000000,NaN,WETH,1.0,2025-04-30 23:59:59+00:00,NaN,NaN


In [9]:
one_destination = dest_df[dest_df["destination_vault_address"] == "0xfB6f99FdF12E37Bfe3c4Cf81067faB10c465fb24"].copy()
one_destination

,owned_shares,destination_vault_address,block,token_address,quantity,underlying_token_total_supply,price_per_share,safe_total_supply,symbol,safe_price,datetime,portion_ownership,proportional_value
2290,0.0,0xfB6f99FdF12E37Bfe3c4Cf81067faB10c465fb24,20759464,0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0,166.398953,460.238347,1.010355,215.402451,wstETH,1.178529,2024-09-15 23:59:59+00:00,0.0,0.0
2291,0.0,0xfB6f99FdF12E37Bfe3c4Cf81067faB10c465fb24,20766617,0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0,168.350693,485.619576,1.010071,240.713725,wstETH,1.178491,2024-09-16 23:59:59+00:00,0.0,0.0
2292,0.0,0xfB6f99FdF12E37Bfe3c4Cf81067faB10c465fb24,20773761,0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0,182.678109,465.797688,1.010095,220.839368,wstETH,1.178273,2024-09-17 23:59:59+00:00,0.0,0.0
2293,0.0,0xfB6f99FdF12E37Bfe3c4Cf81067faB10c465fb24,20780916,0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0,149.670360,465.851561,1.010410,220.839368,wstETH,1.178817,2024-09-18 23:59:59+00:00,0.0,0.0
2294,0.0,0xfB6f99FdF12E37Bfe3c4Cf81067faB10c465fb24,20788084,0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0,190.014568,448.627487,1.010333,220.839368,wstETH,1.178462,2024-09-19 23:59:59+00:00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2743,0.0,0xfB6f99FdF12E37Bfe3c4Cf81067faB10c465fb24,22363838,0xA35b1B31Ce002FBF2058D22F30f95D405200A15b,372.299464,734.388495,1.024108,0.000000,ETHx,1.057706,2025-04-27 23:59:59+00:00,0.0,0.0
2744,0.0,0xfB6f99FdF12E37Bfe3c4Cf81067faB10c465fb24,22370998,0xA35b1B31Ce002FBF2058D22F30f95D405200A15b,372.531344,735.421130,1.024313,0.000000,ETHx,1.057939,2025-04-28 23:59:59+00:00,0.0,0.0
2745,0.0,0xfB6f99FdF12E37Bfe3c4Cf81067faB10c465fb24,22378150,0xA35b1B31Ce002FBF2058D22F30f95D405200A15b,367.295563,735.450235,1.024356,0.000000,ETHx,1.058046,2025-04-29 23:59:59+00:00,0.0,0.0
2746,0.0,0xfB6f99FdF12E37Bfe3c4Cf81067faB10c465fb24,22385293,0xA35b1B31Ce002FBF2058D22F30f95D405200A15b,378.206718,760.005217,1.024143,0.000000,ETHx,1.057777,2025-04-30 23:59:59+00:00,0.0,0.0


In [10]:
import plotly.io as pio

pio.templates.default = None
px.bar(one_destination.pivot(index="datetime", columns="symbol", values="quantity"))

In [17]:
dest_df

,owned_shares,destination_vault_address,block,token_address,quantity,underlying_token_total_supply,price_per_share,safe_total_supply,symbol,safe_price,datetime,portion_ownership,proportional_value
0,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20759464,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7077.280862,13157.451200,1.036523,12302.611124,WETH,1.0,2024-09-15 23:59:59+00:00,0.0,0.0
1,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20766617,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7091.767922,13157.496413,1.036163,12383.288934,WETH,1.0,2024-09-16 23:59:59+00:00,0.0,0.0
2,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20773761,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7042.071304,13157.496413,1.036462,12383.288934,WETH,1.0,2024-09-17 23:59:59+00:00,0.0,0.0
3,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20780916,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7145.366574,13268.458946,1.036491,12479.834854,WETH,1.0,2024-09-18 23:59:59+00:00,0.0,0.0
4,0.000000,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20788084,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,7069.241070,13268.526874,1.036501,12494.174709,WETH,1.0,2024-09-19 23:59:59+00:00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13506,110.472695,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22363838,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,110.472695,NaN,1.000000,NaN,WETH,1.0,2025-04-27 23:59:59+00:00,NaN,NaN
13507,273.545207,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22370998,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,273.545207,NaN,1.000000,NaN,WETH,1.0,2025-04-28 23:59:59+00:00,NaN,NaN
13508,179.975418,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22378150,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,179.975418,NaN,1.000000,NaN,WETH,1.0,2025-04-29 23:59:59+00:00,NaN,NaN
13509,188.715846,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22385293,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,188.715846,NaN,1.000000,NaN,WETH,1.0,2025-04-30 23:59:59+00:00,NaN,NaN


In [16]:
dest_df.groupby("symbol")["underlying_token_total_supply"].max()

symbol
ETHx       2829.985434
OETH      14654.330428
WETH      20863.280056
cbETH       418.232381
frxETH    10764.601942
osETH     20863.280056
pxETH     11687.198527
rETH      14860.440026
stETH      8114.885421
weETH      5117.872032
wstETH    12865.169846
Name: underlying_token_total_supply, dtype: float64

In [ ]:
# 0x40219bBda953ca811d2D0168Dc806a96b84791d9

In [30]:
dest_df[dest_df["symbol"] == "rETH"]

,owned_shares,destination_vault_address,block,token_address,quantity,underlying_token_total_supply,price_per_share,safe_total_supply,symbol,safe_price,datetime,portion_ownership,proportional_value
229,0.0,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20759464,0xae78736Cd615f374D3085123A210448E74Fc6393,5870.693219,13157.451200,1.036523,12302.611124,rETH,1.117537,2024-09-15 23:59:59+00:00,0.0,0.0
230,0.0,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20766617,0xae78736Cd615f374D3085123A210448E74Fc6393,5857.599680,13157.496413,1.036163,12383.288934,rETH,1.116761,2024-09-16 23:59:59+00:00,0.0,0.0
231,0.0,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20773761,0xae78736Cd615f374D3085123A210448E74Fc6393,5902.084057,13157.496413,1.036462,12383.288934,rETH,1.117432,2024-09-17 23:59:59+00:00,0.0,0.0
232,0.0,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20780916,0xae78736Cd615f374D3085123A210448E74Fc6393,5912.023158,13268.458946,1.036491,12479.834854,rETH,1.117600,2024-09-18 23:59:59+00:00,0.0,0.0
233,0.0,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20788084,0xae78736Cd615f374D3085123A210448E74Fc6393,5980.207146,13268.526874,1.036501,12494.174709,rETH,1.117621,2024-09-19 23:59:59+00:00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10987,0.0,0x2B08137BeABd2454AD3631DEB754F97C5c93eB78,22363838,0xae78736Cd615f374D3085123A210448E74Fc6393,120.621615,264.866433,1.023028,263.573764,rETH,1.134202,2025-04-27 23:59:59+00:00,0.0,0.0
10988,0.0,0x2B08137BeABd2454AD3631DEB754F97C5c93eB78,22370998,0xae78736Cd615f374D3085123A210448E74Fc6393,125.404946,264.866433,1.023049,263.573764,rETH,1.134254,2025-04-28 23:59:59+00:00,0.0,0.0
10989,0.0,0x2B08137BeABd2454AD3631DEB754F97C5c93eB78,22378150,0xae78736Cd615f374D3085123A210448E74Fc6393,131.083711,264.866433,1.022783,263.573764,rETH,1.133703,2025-04-29 23:59:59+00:00,0.0,0.0
10990,0.0,0x2B08137BeABd2454AD3631DEB754F97C5c93eB78,22385293,0xae78736Cd615f374D3085123A210448E74Fc6393,130.592694,267.110200,1.022793,263.573764,rETH,1.133719,2025-04-30 23:59:59+00:00,0.0,0.0


TokemakAddress(eth='0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', base='0x4200000000000000000000000000000000000006')

In [ ]:
dest_df

In [27]:
dest_df.groupby("symbol")["proportional_value"].mean().round(2).astype(int)

symbol
ETHx                 361
OETH                  43
WETH      39783378703905
cbETH                  0
frxETH                 0
osETH                 46
pxETH                981
rETH       1238078769398
stETH                637
weETH                 49
wstETH                16
Name: proportional_value, dtype: int64

In [21]:
tvl_in_each_asset_df = (
    dest_df.groupby(["datetime", "symbol"])["proportional_value"]
    .sum()
    .reset_index()
    .pivot(index="datetime", columns="symbol", values="proportional_value")
)
# rETh and WETH are wrong not sure why
px.bar(tvl_in_each_asset_df)

In [12]:
# why is this 3m rows?
# that can't be right, right
# that c